# 第3章: 正規表現

Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ．

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [10]:
import gzip
import json

output_text = ''
with gzip.open('jawiki-country.json.gz', 'r') as f:
    for line in f:
        article = json.loads(line)
        if article['title'] == 'イギリス':
            output_text = article['text']

with open('output/q20.txt', 'w', encoding='utf8') as f:
    f.write(output_text)

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [80]:
import re

with open('output/q20.txt', 'r', encoding='utf8') as f:
    all_lines = [line for line in f]

pattern = re.compile(r'\[{2}Category:(.+)\]{2}')
target_lines = [line for line in all_lines if re.search(pattern, line)]

with open('output/q21.txt', 'w', encoding='utf8') as f:
    f.writelines(target_lines)

## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [83]:
import re

with open('output/q20.txt', 'r', encoding='utf8') as f:
    all_lines = [line for line in f]

pattern = re.compile(r'\[{2}Category:(.+)\]{2}')
target_lines = [line.replace('[[', '').replace('Category:','').replace('|*', '').replace(']]', '').replace('|元', '') for line in all_lines if re.search(pattern, line)]

with open('output/q22.txt', 'w', encoding='utf8') as f:
    f.writelines(target_lines)

## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [115]:
import re

with open('output/q20.txt', 'r', encoding='utf8') as f:
    all_lines = [line for line in f]

pattern = re.compile(r'^=+.*=+$')
target_texts = [line.replace('=', '').rstrip() + '\t' + str(line.count('=') // 2 - 1) for line in all_lines if re.search(pattern, line)]

with open('output/q23.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(target_texts))

## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [116]:
import re

with open('output/q20.txt', 'r', encoding='utf8') as f:
    all_lines = [line for line in f]

pattern = re.compile(r'ファイル:(.+?)\|')
target_texts = [re.findall(pattern, line)[0] for line in all_lines if re.findall(pattern, line)]

with open('output/q24.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(target_texts))

## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [146]:
import re
import json

with open('output/q20.txt', 'r', encoding='utf8') as f:
    all_lines = [line for line in f]

pattern = re.compile(r'\|(.+)\s=\s*(.+)')
target_fields = {}
for line in all_lines:
    search_result = re.search(pattern, line)
    if search_result:
        target_fields[search_result.groups()[0]] = search_result.groups()[1]

with open('output/q25.json', 'w', encoding='utf8') as f:
    json.dump(target_fields, f, indent=4)

## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [172]:
import re
import json

with open('output/q20.txt', 'r', encoding='utf8') as f:
    all_lines = [line for line in f]

pattern = re.compile(r'\|(.+)\s=\s*(.+)')
pattern_emphasis = re.compile(r'\'{2,5}(.+?)\'{2,5}')
target_fields = {}
for line in all_lines:
    search_result = re.search(pattern, line)
    if search_result:
        value = search_result.groups()[1]
        value = re.sub(pattern_emphasis, '\\1', value) # マッチ結果から、強調マークアップを除去
        target_fields[search_result.groups()[0]] = value

with open('output/q26.json', 'w', encoding='utf8') as f:
    json.dump(target_fields, f, indent=4)

## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: [マークアップ早見表）](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)．

In [27]:
import re
import json

with open('output/q20.txt', 'r', encoding='utf8') as f:
    all_lines = [line for line in f]

pattern = re.compile(r'\|(.+)\s=\s*(.+)')
pattern_emphasis = re.compile(r'\'{2,5}(.+?)\'{2,5}')
pattern_inner_link = re.compile(r'\[\[(?:[^|]*?\|)??([^|]*?)\]\]')
pattern_inner_link2 = re.compile(r'\[\[(?:[^|]*?\|)(\{\{(?:.*?)\}\}*?)\]\]') # 表示文字の箇所にlangテンプレートを含むパターン（国歌フィールド）
target_fields = {}
for line in all_lines:
    search_result = re.search(pattern, line)
    if search_result:
        value = search_result.groups()[1]
        value = re.sub(pattern_emphasis, '\\1', value) # マッチ結果から、強調マークアップを除去
        value = re.sub(pattern_inner_link, '\\1', value) # 内部リンクマークアップを除去
        value = re.sub(pattern_inner_link2, '\\1', value) # 内部リンクマークアップを除去
        target_fields[search_result.groups()[0]] = value

with open('output/q27.json', 'w', encoding='utf8') as f:
    json.dump(target_fields, f, indent=4)

## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [15]:
import re
import json

with open('output/q20.txt', 'r', encoding='utf8') as f:
    all_lines = [line for line in f]

pattern = re.compile(r'\|(.+)\s=\s*(.+)')
pattern_emphasis = re.compile(r'\'{2,5}(.+?)\'{2,5}')
pattern_inner_link = re.compile(r'\[\[(?:[^|]*?\|)??([^|]*?)\]\]')
pattern_inner_link2 = re.compile(r'\[\[(?:[^|]*?\|)(\{\{(?:.*?)\}\}*?)\]\]') # 表示文字の箇所にlangテンプレートを含むパターン（国歌フィールド）
pattern_media_file = re.compile(r'\[\[(?:[^|]*?\|)*?([^|]*?)\]\]')
pattern_template_lang = re.compile(r'\{\{lang(?:[^|]*?\|)*?([^|]*?)\}\}')
pattern_template_tmp_link = re.compile(r'\{\{仮リンク\|(.*?)(?:\|.*?)*?\}\}')
pattern_template_citeweb = re.compile(r'\{\{Cite web(?:|\s)\|(?:[^|]*?)*?(?:\|title=(.*?))(?:\|[^|]*?)*?\}\}')
pattern_template_center = re.compile(r'\{\{center\|(.*?)\}\}') # ファイルより先に除去（centerの表示文字の箇所にファイルが入っているため）
pattern_outer_link = re.compile(r'\[http(?:|s):\/\/(?:[^\s]*?\s)([^\]]*?)\]')
pattern_br_ref_html = re.compile(r'<\/?[br|ref][^>]*?>')
target_fields = {}
for line in all_lines:
    search_result = re.search(pattern, line)
    if search_result:
        value = search_result.groups()[1]
        value = re.sub(pattern_emphasis, '\\1', value) # マッチ結果から、強調マークアップを除去
        value = re.sub(pattern_inner_link, '\\1', value) # 内部リンクマークアップを除去
        value = re.sub(pattern_inner_link2, '\\1', value) # 内部リンクマークアップを除去
        # MediaWikiマークアップを除去
        value = re.sub(pattern_media_file, '\\1', value) # ファイル
        value = re.sub(pattern_template_lang, '\\1', value) # langテンプレート
        value = re.sub(pattern_template_tmp_link, '\\1', value) # 仮リンクテンプレート
        value = re.sub(pattern_template_citeweb, '\\1', value) # Citewebテンプレート（title以外を除去）
        value = re.sub(pattern_template_center, '\\1', value) # centerテンプレート
        value = re.sub(pattern_outer_link, '\\1', value) # 外部リンク
        value = re.sub(pattern_br_ref_html, '', value) # br, refタグ
        target_fields[search_result.groups()[0]] = value

with open('output/q28.json', 'w', encoding='utf8') as f:
    json.dump(target_fields, f, indent=4)

## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: [MediaWiki API](http://www.mediawiki.org/wiki/API:Main_page/ja)の[imageinfo](https://www.mediawiki.org/wiki/API:Imageinfo)を呼び出して，ファイル参照をURLに変換すればよい）

In [16]:
import json
import requests

with open('output/q28.json', 'r', encoding='utf8') as f:
    national_flag_name = json.load(f)['国旗画像']

session = requests.Session()
URL = 'https://www.mediawiki.org/w/api.php'
PARAMS = {
    'action': 'query',
    'format': 'json',
    'prop': 'imageinfo',
    'titles': 'File:' + national_flag_name,
    'iiprop': 'url'
}

request = session.get(url=URL, params=PARAMS)
data = request.json()
pages = data['query']['pages']
image_urls = [page['imageinfo'][0]['url'] for page in pages.values()]
print(image_urls)

with open('output/q29.txt', 'w', encoding='utf8') as f:
    f.writelines(image_urls)

['https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg']
